In [213]:
import numpy as np
import pandas as pd

import edhec_risk_kit_203 as erk

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [214]:
# 49 ind 
def get_ind_file(filetype, ew=False):
    """
    Load and format the Ken French 49 Industry Portfolios files
    """
    divisor = 1
    if filetype == "returns":
        filetype = "ew_rets" if ew else "vw_rets"
        divisor = 100
                         
    ind = pd.read_csv(f"data/ind49_m_{filetype}.csv", header=0, index_col=0)/divisor
    ind.index = pd.to_datetime(ind.index, format="%Y%m").to_period('M')
    ind.columns = ind.columns.str.strip()
    return ind

def get_ind_returns(ew=False):
    """
    Load and format the Ken French 49 Industry Portfolios Value Weighted Monthly Returns
    """
    return get_ind_file("returns", ew=ew)

def get_ind_nfirms():
    """
    Load and format the Ken French 49 Industry Portfolios Average number of Firms
    """
    return get_ind_file("nfirms")

In [215]:
rets = get_ind_returns()
ff = erk.get_fff_returns()

What is the CAPM (Single Factor) **Beta** 
when evaluated over the entire period (1991-2018) of Beer?

In [219]:
import statsmodels.api as sm

In [232]:
def capm(ind, t0=None, t1=None):
    y = rets[[ind]][t0:t1] - ff[['RF']][t0:t1].values
    x = sm.add_constant(ff[['Mkt-RF']][t0:t1])
    return sm.OLS(y, x).fit()

def famafrench(ind, t0=None, t1=None):
    y = rets[[ind]][t0:t1] - ff[['RF']][t0:t1].values
    x = sm.add_constant(ff[['Mkt-RF', 'SMB', 'HML']][t0:t1])
    return sm.OLS(y, x).fit()

def coefs(ind, fun, t0=None, t1=None, name=None):
    m = fun(ind, t0, t1)
    return m.params[name] if name else m.params


In [235]:
coefs('Beer',capm,'1991', '2018','Mkt-RF')

0.5295417263363025

In [236]:
coefs('Steel',capm,'1991', '2018','Mkt-RF')

1.5546095998085288

In [238]:
coefs('Beer',capm,'2013', '2018','Mkt-RF')

0.585960389056597

In [239]:
coefs('Steel',capm,'2013', '2018','Mkt-RF')

1.4169446523353806

 which of the 49 industries had the highest CAPM Beta when evaluated over the 1991-1993 (both included) period?

In [246]:
inds = rets.columns.to_series()
betas = inds.apply(coefs, t0='1991', t1='1993', fun=capm)
betas.head()

,const,Mkt-RF
Agric,-0.002842,1.028381
Food,-0.008876,1.262609
Soda,-0.006793,1.319035
Beer,-0.000163,1.198078
Smoke,-0.018946,1.323776


In [250]:
# 5. highest CAPM Beta over the 1991-1993 (both included) period?
betas['Mkt-RF'].idxmax()

'Hlth'

In [252]:
# 6. lowest  CAPM Beta over the 1991-1993 (both included) period?
betas['Mkt-RF'].idxmin()

'Gold'

# 7, 8, 9, 10

7. highest Small Cap tilt when analyzed over the entire 1991-2018 period?

In [261]:
fftilts = inds.apply(coefs, t0='1991', t1='2018',fun=famafrench)
fftilts['SMB'].idxmax()

'FabPr'

8. highest Large Cap tilt when analyzed over the period as the previous? 

In [258]:
fftilts['SMB'].idxmin()

'Beer'

9. highest Value tilt when analyzed over the entire period?    

In [259]:
fftilts['HML'].idxmax()

'Txtls'

10. highest Growth tilt when analyzed over the entire period?    

In [260]:
fftilts['HML'].idxmin()

'Softw'

In [263]:
#Load the "ind49_m_vw_rets.csv" file.
fortynine = pd.read_csv("data/ind49_m_vw_rets.csv", parse_dates=True, index_col=0)/100
# Select the period from 1991 to 2018 inclusive.
fortynine91to18=fortynine.loc["1991":"2019"].reset_index(drop=True)
# The Fama French data should have already been loaded if you run
# the notebook "lab_203" from the beginning.
# So, just go ahead and select the period.
mkt_excess91to18 = fff.loc["1991":"2019"].reset_index(drop=True)

result = erk.regress(fortynine91to18, mkt_excess91to18)
params=result.params
params.columns=fortynine91to18.columns

In [265]:
params.T.sort_values(by="SMB",ascending=False)

,Mkt-RF,SMB,HML,RF,Alpha
FabPr,1.047132,0.686027,0.496060,-0.356768,-0.001487
RlEst,1.150799,0.680132,0.912121,-3.006597,0.001763
Coal,1.110073,0.544504,0.476809,6.541754,-0.012814
Txtls,1.236776,0.529438,0.997089,-2.568338,0.002357
Steel,1.509318,0.494700,0.315190,1.760300,-0.008361
LabEq,1.177370,0.465954,-0.262712,1.424596,0.000206
Rubbr,0.982567,0.445960,0.471812,-0.287894,0.002465
Cnstr,1.191184,0.407341,0.611075,0.138247,-0.001242
Mach,1.280647,0.351694,0.306406,-0.066830,0.000598
Toys,0.940096,0.326875,0.429956,-0.633803,0.000231
